# Lunar Phases and Cryptocurrency Returns

We obtain preliminary results investigating the relation between lunar phases and cryptocurrency returns. Our findings show that trading guided by  moon phases — namely, buying at a full moon phase and selling at the next new moon phase — shows significantly higher returns than other baseline methods. Further, we show an even stronger sinusoidal relationship between the moon phase in which we begin two-week trading (approximate length of half a moon cycle) and cryptocurrency market returns. We show that highest returns are achieved when we buy at a full moon phase and sell at a new moon phase, implying these phases map to local bottoms and tops, [something that has long been hypothesized](https://twitter.com/marenaltman/status/1427292474523881483?s=20&t=NhDoyOIzXYU75ZmdoHPo8w).

In [246]:
import bisect
import glob
import numpy as np
import pandas as pd
import pylunar
import re
import seaborn as sns
import statsmodels.formula.api as smf
from tqdm import tqdm
from matplotlib import pyplot as plt

pd.options.mode.chained_assignment = None
pd.options.display.float_format = '{:,.3f}'.format
pd.set_option("display.precision", 4)

In [129]:
SECONDS_IN_DAY = 86400

## Collect data
We consider the top 30 cryptocurrency tokens, by marketcap, listed on CoinGecko on Aug. 14, 2022, not including stablecoins or wrapped tokens. The data was gathered using the [CoinGecko API](https://www.coingecko.com/en/api). The script used to call the API is in this directory `./collect_data.py`.

In [130]:
DF_MAIN = pd.concat([pd.read_csv(fname) for fname in glob.glob("./data/CoinGecko_*.csv")])

Clean up data.

We remove rows where either marketcap is zero or total volume is zero. This usually indicates to very early data, near a tokens listing.

In [131]:
# clean ticker symbol
DF_MAIN['ticker'] = [tkr.split('|')[0] for tkr in DF_MAIN['ticker']]

for col in ['market_caps', 'total_volumes']:
    print('Removing {} rows where `{}` == 0'.format(
        (DF_MAIN[col] == 0).sum(), col
    ))
    DF_MAIN = DF_MAIN[DF_MAIN[col] != 0]

# get latest marketcaps and sort tickers by that,
# preseving unixtime sorting
custom_dict = {
    v: k
    for (k, v) in enumerate(
        DF_MAIN.groupby('ticker').last().total_volumes.sort_values(ascending=False).index
    )
}
DF_MAIN = DF_MAIN.sort_values(
    by='ticker',
    key=lambda x: x.map(custom_dict)
).groupby('ticker', sort=False).apply(
    lambda x: x.sort_values('unixtime')
).reset_index(drop=True)

Removing 443 rows where `market_caps` == 0
Removing 637 rows where `total_volumes` == 0


## Summary Statistics

In [132]:
def cumulative_return(x):
    return (x.iloc[-1] - x.iloc[0]) / x.iloc[0]

def mean_monthly_log_return(x):
    return np.mean(np.log(x.shift(-30)/x).iloc[:-30])

def std_monthly_log_return(x):
    return np.std(np.log(x.shift(-30)/x).iloc[:-30])

df = DF_MAIN.groupby("ticker").agg(
    start_date=('date', lambda x: x.values[0][:10]),
    no_obs=('date', lambda x: len(x)),
    cumulative_return=('prices', cumulative_return),
    mean_monthly_log_return=('prices', mean_monthly_log_return),
    std_monthly_log_return=('prices', std_monthly_log_return),
).sort_values('start_date')
df.style.format(formatter={
    "cumulative_return": "{:,.0%}",
    "mean_monthly_log_return": "{:.3f}",
    "std_monthly_log_return": "{:.2f}"
})

,start_date,no_obs,cumulative_return,mean_monthly_log_return,std_monthly_log_return
ticker,,,,,
XRP,2013-12-27,3139,"1,364%",0.026,0.45
LTC,2013-12-27,3138,169%,0.008,0.33
BTC,2013-12-27,3138,"3,121%",0.031,0.23
DOGE,2013-12-27,3137,"12,707%",0.047,0.47
XMR,2014-05-21,2990,"6,198%",0.042,0.37
XLM,2014-08-06,2911,"4,386%",0.040,0.46
ETH,2015-08-08,2549,"127,406%",0.083,0.39
ETC,2016-07-25,2198,"6,278%",0.033,0.40
BCH,2017-08-02,1825,-81%,-0.022,0.41


### Add moon phase data

In [133]:
mi = pylunar.MoonInfo((51, 30, 36), (0, 7, 5)) # London, UK

def moon_phase_data(df):

    def moon_phase(date_string):
        # example data string:
        # "2012-06-19 14:08:00"
        mi.update(tuple(map(int, re.split("-|\s+|:", date_string))))
        return mi.fractional_phase()

    moon_phases = df['date'].apply(moon_phase)
    s = (moon_phases.shift(1) > moon_phases) & (moon_phases.shift(-1) > moon_phases)
    new_moon_idxs = s.index[s]

    def moon_cycle(idx):
        return bisect.bisect(new_moon_idxs, idx)

    def day_after_new_moon(idx):
        nearest_new_moon = new_moon_idxs[moon_cycle(idx) - 1]
        result = idx - nearest_new_moon
        if result >= 0:
            return result
        return 30 - (new_moon_idxs[0] - idx)

    s1 = df.apply(lambda row: moon_cycle(row.name),axis=1)
    s2 = df.apply(lambda row: day_after_new_moon(row.name), axis=1)
    
    return pd.DataFrame({'moon_cycle': s1, 'days_after_new_moon': s2})

In [134]:
DF_MAIN = DF_MAIN.join(DF_MAIN.groupby('ticker').apply(moon_phase_data))
DF_MAIN.tail()

,unixtime,date,ticker,prices,market_caps,total_volumes,moon_cycle,days_after_new_moon
47775,1658880000,2022-07-27 02:00:00,LEO,5.084,"4,764,746,448.096","1,970,945.047",39,28
47776,1658966400,2022-07-28 02:00:00,LEO,5.178,"4,843,044,935.433","1,707,115.191",39,29
47777,1659052800,2022-07-29 02:00:00,LEO,5.321,"4,965,723,084.313","1,898,325.579",40,0
47778,1659139200,2022-07-30 02:00:00,LEO,5.181,"4,848,771,860.852","2,088,763.614",40,1
47779,1659225600,2022-07-31 02:00:00,LEO,5.056,"4,716,690,886.729","1,955,159.439",40,2


## Test for relationship between moon phase and returns

- We look at n-day trading (where n is 15, 30, 60) and ... we call this a cycle

- We shift the start of the cycle by one day

- We pick 30 & 60 since these correspond to buying and selling on the same moon phase. And if there is a moon phase effect, using these days should counteract any effect for any other n day period (e.g. 7, 14).

In [ ]:
def cumulative_return(row, df, days):
    #print(row)
    ts_start = row.unixtime
    ts_end = ts_start + days * SECONDS_IN_DAY
    if ts_end not in df.index:
        return np.nan
    row2 = df.loc[ts_end]
    #print(ts_start, ts_end, row.prices, row2.prices)
    return (row2.prices - row.prices) / row.prices

def mean_daily_log_return(row, df, days):
    ts_start = row.unixtime
    ts_end = ts_start + days * SECONDS_IN_DAY
    # get daily log returns in this period
    log_returns = []
    tss = list(range(ts_start, ts_end + SECONDS_IN_DAY, SECONDS_IN_DAY))
    tss = [ts for ts in tss if ts in set(df.index)]
    try:
        s = df.loc[tss].prices
        return (np.log(s.shift(-1) / s).iloc[:-1]).mean()
    except KeyError:
        return np.nan
    
def compute_returns(df):
    # remove rows where `days_after_new_moon` >= 29 since data is much sparser for these values
    df = df[df.days_after_new_moon <= 28]
    # cosine transform
    df["cos_trans"] = np.cos(2 * np.pi * df["days_after_new_moon"] / 29.53)

    for days in tqdm([15, 30, 60]):
        for fn in [cumulative_return, mean_daily_log_return]:
            col = '{}_day_{}'.format(days, fn.__name__)
            df[col] = df.apply(
                lambda row: fn(row, df.set_index("unixtime"), days),
                axis=1
            )    

    return df

In [314]:
DF_RETURNS = DF_MAIN.groupby('ticker').apply(compute_returns)

100%|█████████████████████████████████████████████| 3/3 [01:56<00:00, 38.84s/it]


In [ ]:
#DF_RETURNS.head(30)

In [315]:
# initialize DataFrame
DF_RESULT = pd.DataFrame([],
    index=pd.Index(tickers, name='Ticker:'),
    columns=pd.MultiIndex.from_product([
      ['15 Day', '30 Day', '60 Day'],
      ['Cumulative', 'Mean Daily Log'],
      #['Cumulative', 'Mean Daily Log'],
      ['coef', 't', 'P>|t|']
    ], names=['Window:', 'Return:', 'Metric:'])
)

In [316]:
DF_RETURNS = DF_RETURNS.reset_index(drop=True)

In [302]:
#DF_RETURNS

,unixtime,date,ticker,prices,market_caps,total_volumes,moon_cycle,days_after_new_moon,cos_trans,15_day_cumulative_return,15_day_mean_daily_log_return,30_day_cumulative_return,30_day_mean_daily_log_return,60_day_cumulative_return,60_day_mean_daily_log_return
0,1508284800,2017-10-18 02:00:00,ADA,0.027,"696,021,404.308","2,351,678.122",0,28,0.947,-0.206,NaN,-0.035,NaN,NaN,NaN
1,1508457600,2017-10-20 02:00:00,ADA,0.030,"785,580,027.642","8,883,473.082",1,0,1.000,-0.280,-0.022,-0.085,-0.003,17.080,NaN
2,1508544000,2017-10-21 02:00:00,ADA,0.029,"741,202,123.880","5,308,856.917",1,1,0.977,-0.230,-0.017,0.049,0.002,16.220,NaN
3,1508630400,2017-10-22 02:00:00,ADA,0.028,"720,669,848.472","2,901,876.122",1,2,0.911,-0.198,-0.015,0.029,0.001,16.914,NaN
4,1508716800,2017-10-23 02:00:00,ADA,0.026,"675,273,816.053","3,505,126.543",1,3,0.803,-0.144,-0.010,0.103,0.003,14.919,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46882,1658793600,2022-07-26 02:00:00,XRP,0.337,"16,267,799,275.739","12,266,428,974.382",106,27,0.859,NaN,NaN,NaN,NaN,NaN,NaN
46883,1658880000,2022-07-27 02:00:00,XRP,0.337,"16,280,339,620.383","28,749,929,057.470",106,28,0.947,NaN,NaN,NaN,NaN,NaN,NaN
46884,1659052800,2022-07-29 02:00:00,XRP,0.372,"17,993,565,318.244","15,143,779,787.628",107,0,1.000,NaN,NaN,NaN,NaN,NaN,NaN
46885,1659139200,2022-07-30 02:00:00,XRP,0.370,"18,046,047,855.846","9,366,962,075.838",107,1,0.977,NaN,NaN,NaN,NaN,NaN,NaN


In [317]:
tickers = DF_RETURNS.groupby('ticker').last().sort_values('market_caps', ascending=False).index

for i,tkr in tqdm(enumerate(tickers), total=len(tickers)):
    #print(tkr)
    df = DF_RETURNS[DF_RETURNS.ticker == tkr]
    #print(DF_MAIN.ticker)
    df = df.reset_index(drop=True)
    
    #print(df)
    
    # remove rows where `days_after_new_moon` >= 29 since data is much sparser for these values
    df = df[df.days_after_new_moon <= 28]
    # cosine transform
    cos_trans = np.cos(2 * np.pi * df["days_after_new_moon"] / 29.53)
    
    regs = []
    for days in [15, 30, 60]:
        for r in ['cumulative_return', 'mean_daily_log_return']:
            key = '{}_day_{}'.format(days, r)
            regs.append(
                smf.ols(
                    'Q("{}") ~ cos_trans'.format(key),
                    data=df
                ).fit()
            )
            
    #print(regs[0].params, reg[0].tvalues, reg[0].pvalues)
    
    DF_RESULT.loc[tkr] = [
        x for reg in regs
        for x in [
            reg.params.cos_trans,
            reg.tvalues.cos_trans,
            reg.pvalues.cos_trans
        ]
    ]
    
#DF_RESULT

100%|███████████████████████████████████████████| 30/30 [00:00<00:00, 32.16it/s]


In [318]:
DF_RESULT

Window:     15 Day                                              30 Day         \
Return: Cumulative              Mean Daily Log              Cumulative          
Metric:       coef      t P>|t|           coef      t P>|t|       coef      t   
Ticker:                                                                         
BTC         -0.001 -0.156 0.876         -0.000 -0.205 0.837      0.001  0.152   
ETH         -0.020 -2.261 0.024         -0.001 -2.559 0.011      0.012  0.759   
BNB          0.225  2.701 0.007          0.002  1.797 0.072      0.123  1.244   
XRP          0.017  1.231 0.218          0.000  0.467 0.640     -0.035 -1.214   
ADA          0.024  1.408 0.159          0.001  0.753 0.451      0.036  0.700   
SOL         -0.103 -5.200 0.000         -0.006 -5.277 0.000     -0.014 -0.347   
DOT         -0.063 -4.073 0.000         -0.004 -4.674 0.000      0.010  0.327   
DOGE        -0.028 -1.720 0.086         -0.001 -1.531 0.126      0.019  0.723   
MATIC        0.029  1.042 0.297         -0.000 -0.186 0.852      0.060  1.441   
STETH       -0.127 -9.627 0.000         -0.008 -9.509 0.000     -0.007 -0.356   
SHIB        -0.098 -2.378 0.018         -0.005 -2.863 0.004      0.051  0.506   
AVAX        -0.086 -2.775 0.006         -0.005 -3.833 0.000      0.000  0.003   
TRX         -0.115 -2.787 0.005         -0.001 -1.672 0.095     -0.227 -1.502   
ETC         -0.051 -4.663 0.000         -0.002 -2.504 0.012      0.022  1.174   
LEO          0.004  0.658 0.510         -0.000 -0.258 0.796      0.005  0.665   
OKB         -0.014 -1.271 0.204         -0.001 -1.660 0.097      0.009  0.553   
LTC         -0.004 -0.557 0.577         -0.000 -0.805 0.421      0.006  0.513   
FTT         -0.042 -4.347 0.000         -0.002 -4.300 0.000      0.008  0.473   
UNI         -0.088 -4.949 0.000         -0.006 -5.529 0.000     -0.018 -0.510   
LINK        -0.013 -1.086 0.277         -0.001 -1.417 0.157      0.010  0.479   
CRO          0.007  0.597 0.551         -0.000 -0.282 0.778      0.005  0.244   
NEAR        -0.073 -3.711 0.000         -0.005 -3.969 0.000      0.013  0.359   
ATOM        -0.042 -3.580 0.000         -0.003 -3.938 0.000     -0.002 -0.091   
XLM          0.040  2.110 0.035          0.000  0.118 0.906     -0.017 -0.486   
XMR         -0.014 -1.478 0.140         -0.000 -0.835 0.404      0.007  0.460   
BCH         -0.030 -2.645 0.008         -0.002 -2.892 0.004      0.012  0.685   
ALGO        -0.050 -4.088 0.000         -0.003 -3.625 0.000      0.012  0.670   
FLOW        -0.134 -6.483 0.000         -0.009 -7.057 0.000     -0.006 -0.174   
VET         -0.011 -0.927 0.354         -0.001 -1.246 0.213      0.010  0.594   
FIL         -0.082 -4.077 0.000         -0.004 -3.202 0.001     -0.007 -0.187   

Window:                                       60 Day               \
Return:       Mean Daily Log              Cumulative                
Metric: P>|t|           coef      t P>|t|       coef      t P>|t|   
Ticker:                                                             
BTC     0.879          0.000  0.092 0.926      0.001  0.096 0.924   
ETH     0.448         -0.000 -0.160 0.873      0.034  0.865 0.387   
BNB     0.214          0.000  0.111 0.912      0.772  1.929 0.054   
XRP     0.225          0.000  0.126 0.900     -0.112 -1.231 0.219   
ADA     0.484         -0.000 -0.043 0.966     -0.080 -0.523 0.601   
SOL     0.728          0.000  0.352 0.725     -0.064 -0.675 0.500   
DOT     0.744          0.001  0.754 0.451      0.024  0.348 0.728   
DOGE    0.470          0.000  0.276 0.782      0.061  1.099 0.272   
MATIC   0.150          0.001  1.175 0.240      0.037  0.405 0.686   
STETH   0.722         -0.001 -0.834 0.405     -0.025 -0.889 0.374   
SHIB    0.613          0.000  0.372 0.710      0.149  0.946 0.345   
AVAX    0.998          0.000  0.141 0.888      0.042  0.320 0.749   
TRX     0.133          0.000  0.005 0.996      0.267  1.464 0.143   
ETC     0.240          0.000  0.836 0.403      0.035  0.914 0.361   
LEO

In [319]:
# TODO: toggle cell code

# Just styling stuff; code hidden by default

multiindex = pd.MultiIndex.from_product([
      ['15 Day', '30 Day', '60 Day'],
      ['Cumulative', 'Mean Daily Log'],
      ['coef', 't', 'P>|t|']
])

s = DF_RESULT.style.format(formatter={
    t:(
        "{:.1f}" if t[2] == 't'
        else "{:.4f}" if t[2] == 'P>|t|'
        else "{:.4f}" if t[1] == 'Mean Daily Log'
        else "{:.2f}"
    )
    for t in multiindex
})

s.columns = pd.MultiIndex.from_product([
  ['15 Day', '30 Day', '60 Day'],
  ['Cumulative', 'Mean Daily Log'],
  ['coef', 't', 'P>|t|']
], names=['Window:', 'Return:', ''])

s.set_table_styles([
    {'selector': '.index_name', 'props': 'font-weight:normal; font-weight: normal;'},
    {'selector': 'th.row_heading', 'props': 'font-weight:bold; text-align: center;'},
    {'selector': 'th.col_heading', 'props': 'text-align: center;'},
    {'selector': 'th.col_heading.level0', 'props': 'font-size: 1.5em; border-bottom: 1px solid darkgrey;'},
    {'selector': 'th.col_heading.level1', 'props': 'font-size: 1.2em; border-bottom: 1px solid darkgrey;'},
    {'selector': 'th.col_heading.level2', 'props': 'font-size: 1.2em; border-bottom: 1px solid darkgrey;'},
    {'selector': 'td', 'props': 'text-align: center; font-weight: normal;'},
    {'selector': 'th:not(.index_name)', 'props': 'background-color: black; color: white;'}
])

s.set_table_styles({
    ('30 Day', 'Cumulative', 'coef'): [
        {'selector': 'th', 'props': 'border-left: 2px solid white'},
        {'selector': 'td', 'props': 'border-left: 2px solid black'}
    ],
    ('15 Day', 'Mean Daily Log', 'coef'): [
        {'selector': 'td', 'props': 'border-left: 1px solid black'}
    ],
    ('30 Day', 'Mean Daily Log', 'coef'): [
        {'selector': 'td', 'props': 'border-left: 1px solid black'}
    ]
}, overwrite=False, axis=0)

def highlight_pvalues(s):
    def _color(pvalue):
        if pvalue < 0.001:
            return "FCF947"
        if pvalue < 0.01:
            return "FDFA75"
        if pvalue < 0.05:
            return "FEFDBA"
        if pvalue < 0.1:
            return "FFFEE8"
        else:
            return ""
    props = []
    for x in ['15 Day', '30 Day', '60 Day']:
        for y in ['Cumulative', 'Mean Daily Log']:
            pvalue = s[x, y, 'P>|t|']
            props.extend(['background-color:#{}'.format(_color(pvalue))] * 3)
    return props

s.apply(highlight_pvalues, axis=1)

In [109]:
DF_RESULT

Window:     15 Day                  30 Day                  60 Day         \
Return: Cumulative              Cumulative              Cumulative          
Metric:       coef      t P>|t|       coef      t P>|t|       coef      t   
Ticker:                                                                     
BTC         -0.001 -0.156 0.876      0.001  0.152 0.879      0.001  0.096   
ETH         -0.020 -2.261 0.024      0.012  0.759 0.448      0.034  0.865   
BNB          0.225  2.701 0.007      0.123  1.244 0.214      0.772  1.929   
XRP          0.017  1.231 0.218     -0.035 -1.214 0.225     -0.112 -1.231   
ADA          0.024  1.408 0.159      0.036  0.700 0.484     -0.080 -0.523   
SOL         -0.103 -5.200 0.000     -0.014 -0.347 0.728     -0.064 -0.675   
DOT         -0.063 -4.073 0.000      0.010  0.327 0.744      0.024  0.348   
DOGE        -0.028 -1.720 0.086      0.019  0.723 0.470      0.061  1.099   
MATIC        0.029  1.042 0.297      0.060  1.441 0.150      0.037  0.405   
STETH       -0.127 -9.627 0.000     -0.007 -0.356 0.722     -0.025 -0.889   
SHIB        -0.098 -2.378 0.018      0.051  0.506 0.613      0.149  0.946   
AVAX        -0.086 -2.775 0.006      0.000  0.003 0.998      0.042  0.320   
TRX         -0.115 -2.787 0.005     -0.227 -1.502 0.133      0.267  1.464   
ETC         -0.051 -4.663 0.000      0.022  1.174 0.240      0.035  0.914   
LEO          0.004  0.658 0.510      0.005  0.665 0.506      0.001  0.126   
OKB         -0.014 -1.271 0.204      0.009  0.553 0.580      0.025  1.119   
LTC         -0.004 -0.557 0.577      0.006  0.513 0.608      0.005  0.213   
FTT         -0.042 -4.347 0.000      0.008  0.473 0.636      0.018  0.431   
UNI         -0.088 -4.949 0.000     -0.018 -0.510 0.610     -0.011 -0.129   
LINK        -0.013 -1.086 0.277      0.010  0.479 0.632      0.063  1.893   
CRO          0.007  0.597 0.551      0.005  0.244 0.807      0.000  0.002   
NEAR        -0.073 -3.711 0.000      0.013  0.359 0.720      0.044  0.644   
ATOM        -0.042 -3.580 0.000     -0.002 -0.091 0.928      0.010  0.304   
XLM          0.040  2.110 0.035     -0.017 -0.486 0.627     -0.093 -1.149   
XMR         -0.014 -1.478 0.140      0.007  0.460 0.646      0.018  0.832   
BCH         -0.030 -2.645 0.008      0.012  0.685 0.493      0.035  1.108   
ALGO        -0.050 -4.088 0.000      0.012  0.670 0.503      0.029  0.953   
FLOW        -0.134 -6.483 0.000     -0.006 -0.174 0.862     -0.049 -0.984   
VET         -0.011 -0.927 0.354      0.010  0.594 0.552      0.017  0.520   
FIL         -0.082 -4.077 0.000     -0.007 -0.187 0.851     -0.037 -0.481   

Window:        
Return:        
Metric: P>|t|  
Ticker:        
BTC     0.924  
ETH     0.387  
BNB     0.054  
XRP     0.219  
ADA     0.601  
SOL     0.500  
DOT     0.728  
DOGE    0.272  
MATIC   0.686  
STETH   0.374  
SHIB    0.345  
AVAX    0.749  
TRX     0.143  
ETC     0.361  
LEO     0.900  
OKB     0.263  
LTC     0.831  
FTT     0.667  
UNI     0.897  
LINK    0.058  
CRO     0.998  
NEAR    0.520  
ATOM    0.761  
XLM     0.251  
XMR     0.406  
BCH     0.268  
ALGO    0.341  
FLOW    0.326  
VET     0.603  
FIL     0.631

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
ax1.tick_params(labelrotation=90)
ax2.tick_params(labelrotation=90)
fig.suptitle('Coefficients ... TODO')

df = DF_RESULT[('15 Day', 'Cumulative', 'coef')]
g1 = sns.barplot(x=df.index, y=df.values, color='black', ax=ax1);
g1.set_xticklabels([tkr.split('|')[0] for tkr in DF_RESULT.index]);
g1.set(title='Cumulative');

df = DF_RESULT[('15 Day', 'Mean Daily Log', 'coef')]
g2 = sns.barplot(x=df.index, y=df.values, color='black', ax=ax2);
g2.set_xticklabels([tkr.split('|')[0] for tkr in DF_RESULT.index]);
g2.set(title='Mean Daily Log');

#ax2.plot(x, -y)

In [ ]:
# example, KEEP!!!
#sns.lineplot(data=fmri, x="timepoint", y="signal", hue="event")

In [86]:
def cumulative_return(df, days):
    #print(row)
    ts_start = row.unixtime
    ts_end = ts_start + days * SECONDS_IN_DAY
    if ts_end not in df.index:
        return np.nan
    row2 = df.loc[ts_end]
    #print(ts_start, ts_end, row.prices, row2.prices)
    return (row2.prices - row.prices) / row.prices

def mean_daily_log_return(row, df, days):
    ts_start = row.unixtime
    ts_end = ts_start + days * SECONDS_IN_DAY
    # get daily log returns in this period
    log_returns = []
    tss = list(range(ts_start, ts_end + SECONDS_IN_DAY, SECONDS_IN_DAY))
    try:
        s = df.loc[tss].prices
        return (np.log(s.shift(-1) / s).iloc[:-1]).mean()
    except KeyError:
        return np.nan
    
def mean_daily_log_return(df, days, threshold=0.75):
    
    df2 = pd.concat([
        df[['unixtime', 'prices']].iloc[:-1].reset_index(drop=True),
        df[['unixtime', 'prices']].iloc[1:].reset_index(drop=True)
    ], axis=1)
    df2.columns = ['unixtime0', 'prices0', 'unixtime1', 'prices1']

    df2 = df2[df2.unixtime1 - df2.unixtime0 < 172800]

    if df2.shape[0] < days * threshold:
        return np.nan

    return np.log(df2.prices1/df2.prices0).mean()

In [87]:
def compute_returns(df):
    # remove rows where `days_after_new_moon` >= 29 since data is much sparser for these values
    df = df[df.days_after_new_moon <= 28]
    # cosine transform
    df["cos_trans"] = np.cos(2 * np.pi * df["days_after_new_moon"] / 29.53)

    for days in tqdm([15, 30, 60]):
        for fn in [cumulative_return, mean_daily_log_return]:
            col = '{}_day_{}'.format(days, fn.__name__)
            df[col] = df.apply(
                lambda row: fn(row, df.set_index("unixtime"), days),
                axis=1
            )    

    return df

In [88]:
df = DF_MAIN.groupby('ticker').apply(compute_returns)

100%|█████████████████████████████████████████████| 3/3 [00:10<00:00,  3.65s/it]


In [90]:
df

unixtime                 date ticker  prices  \
ticker                                                         
ADA    14299  1508284800  2017-10-18 02:00:00    ADA   0.027   
       14301  1508457600  2017-10-20 02:00:00    ADA   0.030   
       14302  1508544000  2017-10-21 02:00:00    ADA   0.029   
       14303  1508630400  2017-10-22 02:00:00    ADA   0.028   
       14304  1508716800  2017-10-23 02:00:00    ADA   0.026   
...                  ...                  ...    ...     ...   
XRP    6271   1658793600  2022-07-26 02:00:00    XRP   0.337   
       6272   1658880000  2022-07-27 02:00:00    XRP   0.337   
       6274   1659052800  2022-07-29 02:00:00    XRP   0.372   
       6275   1659139200  2022-07-30 02:00:00    XRP   0.370   
       6276   1659225600  2022-07-31 02:00:00    XRP   0.387   

                    market_caps      total_volumes  moon_cycle  \
ticker                                                           
ADA    14299    696,021,404.308      2,351,678.122           0   
       14301    785,580,027.642      8,883,473.082           1   
       14302    741,202,123.880      5,308,856.917           1   
       14303    720,669,848.472      2,901,876.122           1   
       14304    675,273,816.053      3,505,126.543           1   
...                         ...                ...         ...   
XRP    6271  16,267,799,275.739 12,266,428,974.382         106   
       6272  16,280,339,620.383 28,749,929,057.470         106   
       6274  17,993,565,318.244 15,143,779,787.628         107   
       6275  18,046,047,855.846  9,366,962,075.838         107   
       6276  18,686,873,330.208 37,779,483,824.233         107   

              days_after_new_moon  cos_trans  15_day_cumulative_return  \
ticker                                                                   
ADA    14299                   28      0.947                    -0.206   
       14301                    0      1.000                    -0.280   
       14302                    1      0.977                    -0.230   
       14303                    2      0.911                    -0.198   
       14304                    3      0.803                    -0.144   
...                           ...        ...                       ...   
XRP    6271                    27      0.859                       NaN   
       6272                    28      0.947                       NaN   
       6274                     0      1.000                       NaN   
       6275                     1      0.977                       NaN   
       6276                     2      0.911                       NaN   

              15_day_mean_daily_log_return  30_day_cumulative_return  \
ticker                                                                 
ADA    14299                           NaN                    -0.035   
       14301                        -0.022                    -0.085   
       14302                        -0.017                     0.049   
       14303                        -0.015                     0.029   
       14304                        -0.010                     0.103   
...                                    ...                       ...   
XRP    6271                            NaN                       NaN   
       6272                            NaN                       NaN   
       6274                            NaN                       NaN   
       6275                            NaN                       NaN   
       6276                            NaN                       NaN   

              30_day_mean_daily_log_return  60_day_cumulative_return  \
ticker                                                                 
ADA    14299                           NaN                       NaN   
       14301                        -0.003                    17.080   
       14302                         0.002                    16.220   
       14303                         0.001                    16

In [346]:
g = sns.FacetGrid(df, col="ticker", sharey=False, col_wrap=5)
g.map(sns.lineplot, "days_after_new_moon", "30_day_cumulative_return")

KeyError: 'ticker'

In [166]:
def mean_daily_log_return(row, df, days, threshold=.5):
    print(row)
    ts_start = row.unixtime
    ts_end = ts_start + days * SECONDS_IN_DAY
    # get daily log returns in this period
    s = df.prices.loc[ts_start:ts_start + days]
    
    #print(len(s))
    #print('SHIFT')
    #print(s.shift(-1))
    
    df_prices = pd.DataFrame({0: s, 1: s.shift(-1)})
    
    #print(df_prices)
    
    
    log_returns = []
    tss = list(range(ts_start, ts_end + SECONDS_IN_DAY, SECONDS_IN_DAY))
    try:
        s = df.loc[tss].prices
        return (np.log(s.shift(-1) / s).iloc[:-1]).mean()
    except KeyError:
        print(row.name)
        return np.nan

In [173]:
df = DF_MAIN[DF_MAIN.ticker == 'BTC']
df

,unixtime,date,ticker,prices,market_caps,total_volumes,moon_cycle,days_after_new_moon
0,1388102400,2013-12-27 01:00:00,BTC,734.270,"8,944,473,292.000","62,881,800.000",0,25
1,1388188800,2013-12-28 01:00:00,BTC,738.810,"9,002,769,255.000","28,121,600.000",0,26
2,1388275200,2013-12-29 01:00:00,BTC,726.470,"8,855,251,580.000","27,018,300.000",0,27
3,1388361600,2013-12-30 01:00:00,BTC,760.520,"9,270,681,761.000","24,717,100.000",0,28
4,1388448000,2013-12-31 01:00:00,BTC,755.160,"9,205,343,763.000","21,903,500.000",0,29
...,...,...,...,...,...,...,...,...
3133,1658880000,2022-07-27 02:00:00,BTC,"21,235.612","405,692,421,501.527","53,995,620,035.181",106,28
3134,1658966400,2022-07-28 02:00:00,BTC,"22,908.884","437,193,517,021.845","54,498,281,944.452",106,29
3135,1659052800,2022-07-29 02:00:00,BTC,"23,822.068","454,806,622,614.759","55,609,627,980.657",107,0
3136,1659139200,2022-07-30 02:00:00,BTC,"23,847.583","459,609,388,579.555","41,089,335,195.640",107,1


In [185]:
'''
df.apply(
    lambda row: mean_daily_log_return(row, df.set_index("unixtime"), 10),
    axis=1
)
'''

'''
df2 = pd.DataFrame({
    0: df.set_index('unixtime').prices,
    1: df.set_index('unixtime').prices.shift(-1)
}).iloc[:-1]
'''

pd.concat(df[['unixtime', 'prices']], df[['unixtime', 'prices']])

/var/folders/rg/2dllcd5s75d8hpkp5mvxfwmm0000gq/T/ipykernel_26942/3636372901.py:15: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  pd.concat(df[['unixtime', 'prices']], df[['unixtime', 'prices']])


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [230]:
df = DF_MAIN[DF_MAIN.ticker == 'BTC']

df2 = pd.concat([
    df[['unixtime', 'prices']].iloc[:-1].reset_index(drop=True),
    df[['unixtime', 'prices']].iloc[1:].reset_index(drop=True)
], axis=1)

df2.columns = ['unixtime0', 'prices0', 'unixtime1', 'prices1']

#df2['unixtime'].apply(lambda row: row.iloc[1] - row.iloc[0], axis=1)

df2 = df2[df2.unixtime1 - df2.unixtime0 < 172800]

# if df2.shape ...

np.log(df2.prices1/df2.prices0)


0       0.006
1      -0.017
2       0.046
3      -0.007
4       0.017
        ...  
3132   -0.004
3133    0.076
3134    0.039
3135    0.001
3136   -0.008
Length: 3136, dtype: float64

In [284]:
df = DF_MAIN[DF_MAIN.ticker == 'BTC']
days = 7

#def compute_returns(df):

# fill all missing times
ts_start = int(df.iloc[0].unixtime)
ts_end = int(df.iloc[-1].unixtime)

tss = set(range(ts_start, ts_end + SECONDS_IN_DAY, SECONDS_IN_DAY))
missing_ts = tss - set(df2.unixtime)

df2 = df[['unixtime', 'prices']]

for ts in list(missing_ts):
    df2.loc[len(df2.index)] = [ts, np.nan]

df2 = df2.sort_values('unixtime')

df2["daily_log_return"] = np.log(df2.shift(-1).prices / df2.prices) 

def _mean_daily_returns(row, days, threshold=0.75):
    #print(row)
    #print(df2.iloc[row.name: row.name + days].shape[0], days * threshold)
    if df2.iloc[row.name: row.name + days].shape[0] < days * threshold:
        return np.nan
    return df2.iloc[row.name: row.name + days].daily_log_return.mean()
    
pd.DataFrame({
    "cumulative_return": (df2.shift(-days).prices - df2.prices) / df2.prices,
    "mean_daily_log": df2.apply(lambda row: _mean_daily_returns(row, days), axis=1)
})

df2

,unixtime,prices,daily_log_return
0,1388102400,734.270,0.006
1,1388188800,738.810,-0.017
2,1388275200,726.470,0.046
3,1388361600,760.520,-0.007
4,1388448000,755.160,0.017
...,...,...,...
3133,1658880000,"21,235.612",0.076
3134,1658966400,"22,908.884",0.039
3135,1659052800,"23,822.068",0.001
3136,1659139200,"23,847.583",-0.008


In [270]:
(df2.shift(-days).prices - df2.prices) / df2.prices

0      0.124
1      0.149
2      0.266
3      0.231
4      0.094
        ... 
3133     NaN
3134     NaN
3135     NaN
3136     NaN
3137     NaN
Name: prices, Length: 3138, dtype: float64

In [271]:
df2.apply(
    lambda row: df2.iloc[row.name: row.name + days].daily_log_return.mean(),
    axis=1
)

0       0.017
1       0.020
2       0.034
3       0.030
4       0.013
        ...  
3133    0.027
3134    0.011
3135   -0.004
3136   -0.008
3137      NaN
Length: 3138, dtype: float64